In [1]:
from frontend.database import session
from scripts import generate_fc_layer_circuit, benchmark_utils
from scripts.circuit import circuit
from scripts.particles import variables, enc_vec, enc_mat, enc_tensor3
from scripts.reusable_modules import oneb_adder, nb_adder
from tqdm import tqdm
import numpy as np

In [2]:
from scripts.functions import reduce_add
inp = enc_vec(name='in', nb=4)
out = enc_vec(name='out', nb=3)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')

# NB bit adder

In [3]:
import math
nb = 25
depth = int(math.floor(math.log(nb, 2)))
samples=2
inp = enc_vec(name='in', nb=nb)
out = enc_vec(name='out', nb=depth+1)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')
processing_time = np.zeros(samples)
for idx in range(0, samples):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5))
    inp_dict = inp.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE", eval_strategy='parallel')
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = out.get_variables()
    err_str = str(''.join(results['Outputs'][var] for var in out_vars)[::-1]) +" -- "+ str(np.asarray(inp_arr_val).sum())
    assert int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2) == np.asarray(inp_arr_val).sum(), err_str
    print ",".join(map(lambda x: str(x), inp_arr_val))+" has " + str(int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2))+" ones."
print " ".join(["Time taken is" , str(processing_time.mean()),'sec'])

0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0 has 9 ones.
0,1,0,1,0,0,0,1,0,0,1,1,0,0,1,1,0,1,1,0,0,0,1,0,1 has 11 ones.
Time taken is 1.0 sec


# Single output linear layer

In [4]:
from scripts.nn_layer import linear_layer_1d
from tqdm import tqdm
nb = 10
samples = 2
inputs = enc_vec(name='nn_input', nb=nb)
depth = int(math.ceil(math.log(nb*1.0, 2)))
outputs = enc_vec(name='nn_outputs', nb=1)
outputs_sum = enc_vec(name='sum_outputs', nb=depth+1)
weight = np.asarray(list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5)))
layer = linear_layer_1d(name='linear', weight=weight,
                    inputs=inputs, outputs=outputs)
layer_obj = circuit('linear_layer', layer, const_inputs=[])
layer_obj.write_file(filename='./test_linear.sheep')
processing_time = np.zeros(samples)
for idx in tqdm(range(0, samples)):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5))
    inp_dict = inputs.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test_linear.sheep',inputs_file,"bool","TFHE",eval_strategy='parallel')
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = outputs.get_variables()
    xor_val = (weight == np.asarray([x for x in inp_arr_val]))
    err_str = str(''.join(results['Outputs'][var] for var in out_vars)[::-1]) +" -- "+ str(np.asarray(xor_val).sum())
    assert int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2) == int(np.asarray(xor_val).sum()>nb/2), err_str
print " ".join(["Time taken is" , str(processing_time.mean()),'sec'])

100%|██████████| 2/2 [00:01<00:00,  1.12it/s]

Time taken is 0.453885 sec


# Comparing Numbers (Ciphertext with Plaintext)

In [5]:
from scripts.reusable_modules import compare_cp
cp_inp_val = [1, 0, 1, 1, 1, 1, 1]
tgt = 126
comp_inp = enc_vec(name='cp_inp',nb = len(cp_inp_val))
out_vec = enc_vec(name='cp_out', nb=1)
cp_circ = compare_cp(name='cp', inputs=(comp_inp, tgt), outputs=out_vec)
sum_obj = circuit('cp', cp_circ)
sum_obj.write_file(filename='./test_cp.sheep')
inputs_file = benchmark_utils.write_inputs_file(comp_inp.get_input_dict(cp_inp_val))
results = benchmark_utils.run_circuit('./test_cp.sheep',inputs_file,"bool","TFHE",eval_strategy='parallel')
processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
assert (int(results['Outputs'][out_vec.get_variables()[0]]) == int(int(''.join(map(lambda x : str(x), cp_inp_val)),2) >= tgt))
if int(results['Outputs'][out_vec.get_variables()[0]]) == 1:
    print str( int(int(''.join(map(lambda x : str(x), cp_inp_val[::-1])),2)))+"(CP) is greater than "+str(tgt)+str("(PT)")
else:
    print str( int(int(''.join(map(lambda x : str(x), cp_inp_val[::-1])),2)))+"(CP) is smaller than "+str(tgt)+str("(PT)")
print " ".join(["Time taken is" , results['Processing times (s)']['circuit_evaluation'],'sec'])


125(CP) is smaller than 126(PT)
Time taken is 0.35243 sec


In [6]:
from scripts.nn_layer import linear_layer
from tqdm import tqdm
nb = 10
samples = 1
num_out= 3
depth = int(math.ceil(math.log(nb*1.0, 2)))
inputs = enc_mat(name='nn_input'+str(idx)+'_', size= (1, nb))
outputs = enc_mat(name='nn_outputs'+str(idx)+'_', size=(num_out, 1))
weight = (np.random.uniform(size=(num_out, nb))> 0.5).astype(int)
layer = linear_layer(name='linear', weight=weight,
                    inputs=inputs, outputs=outputs)
layer_obj = circuit('linear_layer', layer, const_inputs=[])
layer_obj.write_file(filename='./test_linear.sheep')
processing_time = np.zeros(samples)
for idx in tqdm(range(0, samples)):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb*1)> 0.5))
    inp_dict = inputs.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test_linear.sheep',inputs_file,"bool","TFHE",eval_strategy='parallel')
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = outputs.get_variables()
    xor_val = (weight == np.asarray([x for x in inp_arr_val]))
    #print(xor_val)
    out_bit_vec = np.asarray([int(results['Outputs'][var]) for var in out_vars])
    true_bit_vec = (np.sum(np.asarray(xor_val), axis=1)>nb/2).astype(int)
    assert np.alltrue(out_bit_vec == true_bit_vec)
print(processing_time.mean())

100%|██████████| 1/1 [00:01<00:00,  1.59s/it]

1.0



|Input Dim | Output Dim | Time|
|:---------| :---------- |:---:|
|100       |    10      | 48s |
|500       |    2       | 47s |
|1024      |    10      | 487s|

In [7]:
import numpy as np
from scripts.particles import enc_mat


def im2col(name, x, hh, ww, stride):
    """
    Args:
      x: image matrix to be translated into columns, (C,H,W)
      hh: filter height
      ww: filter width
      stride: stride
    Returns:
      col: (new_h*new_w,hh*ww*C) matrix, each column is a
           cube that will convolve with a filter
           new_h = (H-hh) // stride + 1, new_w = (W-ww) // stride + 1
    """

    c, h, w = x.size
    new_h = (h - hh) // stride + 1
    new_w = (w - ww) // stride + 1
    col = enc_mat(name=name + 'im2col' + str(c * h * w),
                  size=(new_h * new_w, c * hh * ww))
    for i in range(new_h):
        for j in range(new_w):
            for cidx in range(c):
                for a in range(hh):
                    for b in range(ww):
                        col[i * new_w + j,
                            cidx * hh * ww + a * ww + b] =\
                            x[cidx][i * stride + a][j * stride + b]
    return col
def col2im(mul, h_prime, w_prime, C, tgt_out):
    """
      Args:
      mul: (h_prime*w_prime*w,F) matrix,
           each col should be reshaped to
           C*h_prime*w_prime when C>0, or h_prime*w_prime when C = 0
      h_prime: reshaped filter height
      w_prime: reshaped filter width
      C: reshaped filter channel, if 0, reshape the filter to 2D,
         Otherwise reshape it to 3D
    Returns:
      if C == 0: (F,h_prime,w_prime) matrix
      Otherwise: (F,C,h_prime,w_prime) matrix
    """
    F = mul.size[1]
    h_prime, w_prime = tgt_out.size[0], tgt_out.size[1]
    if(C == 1):
        for i in range(0, F):
            for h_idx in range(0, h_prime):
                for w_idx in range(0, w_prime):
                    tgt_out[i, h_idx, w_idx] = mul[h_idx * w_prime + w_idx][i]


In [8]:

s = enc_tensor3(size=(2,5, 5), name='sam')
i2c = im2col(name='im2col',
      x=s, hh=3, ww=3, stride=1)

In [9]:

st_lst = map(lambda x: map(lambda xt: xt[-3:], x), i2c.name_list)
print map(lambda xt: " ".join(xt), st_lst)
i2c_short = np.asarray(map(lambda x: map(lambda xt: map(lambda xst: xst[-3:], xt), x),s.name_list) )
print(i2c_short)

['802 BF2 B5D C75 CF4 CD7 0A1 33C FFB 398 1A6 C2F 5F0 F68 2E2 207 F55 AFF', 'BF2 B5D E29 CF4 CD7 A83 33C FFB 7BE 1A6 C2F B59 F68 2E2 436 F55 AFF 0E4', 'B5D E29 37D CD7 A83 C15 FFB 7BE 0A5 C2F B59 892 2E2 436 5F3 AFF 0E4 014', 'C75 CF4 CD7 0A1 33C FFB 10F 1A9 793 5F0 F68 2E2 207 F55 AFF 096 803 CB9', 'CF4 CD7 A83 33C FFB 7BE 1A9 793 68E F68 2E2 436 F55 AFF 0E4 803 CB9 C23', 'CD7 A83 C15 FFB 7BE 0A5 793 68E 241 2E2 436 5F3 AFF 0E4 014 CB9 C23 BC5', '0A1 33C FFB 10F 1A9 793 C99 83C EAA 207 F55 AFF 096 803 CB9 183 355 BC4', '33C FFB 7BE 1A9 793 68E 83C EAA 8ED F55 AFF 0E4 803 CB9 C23 355 BC4 B8B', 'FFB 7BE 0A5 793 68E 241 EAA 8ED 596 AFF 0E4 014 CB9 C23 BC5 BC4 B8B 0DB']
[[['802' 'BF2' 'B5D' 'E29' '37D']
  ['C75' 'CF4' 'CD7' 'A83' 'C15']
  ['0A1' '33C' 'FFB' '7BE' '0A5']
  ['10F' '1A9' '793' '68E' '241']
  ['C99' '83C' 'EAA' '8ED' '596']]

 [['398' '1A6' 'C2F' 'B59' '892']
  ['5F0' 'F68' '2E2' '436' '5F3']
  ['207' 'F55' 'AFF' '0E4' '014']
  ['096' '803' 'CB9' 'C23' 'BC5']
  ['183' '355' '

In [10]:
out_cnv = enc_tensor3(name='cnv_out', size=(3, 3, 3))
mul = enc_mat(name='cnv_lin_out', size=(9, 3))
col2im(mul=mul, h_prime=3, w_prime=3, C=1, tgt_out=out_cnv)

st_lst = map(lambda x: map(lambda xt: xt[-3:], x), i2c.name_list)
print map(lambda xt: " ".join(xt), st_lst)
i2c_short = np.asarray(map(lambda x: map(lambda xt: map(lambda xst: xst[-3:], xt), x),out_cnv.name_list) )
print(i2c_short)

['802 BF2 B5D C75 CF4 CD7 0A1 33C FFB 398 1A6 C2F 5F0 F68 2E2 207 F55 AFF', 'BF2 B5D E29 CF4 CD7 A83 33C FFB 7BE 1A6 C2F B59 F68 2E2 436 F55 AFF 0E4', 'B5D E29 37D CD7 A83 C15 FFB 7BE 0A5 C2F B59 892 2E2 436 5F3 AFF 0E4 014', 'C75 CF4 CD7 0A1 33C FFB 10F 1A9 793 5F0 F68 2E2 207 F55 AFF 096 803 CB9', 'CF4 CD7 A83 33C FFB 7BE 1A9 793 68E F68 2E2 436 F55 AFF 0E4 803 CB9 C23', 'CD7 A83 C15 FFB 7BE 0A5 793 68E 241 2E2 436 5F3 AFF 0E4 014 CB9 C23 BC5', '0A1 33C FFB 10F 1A9 793 C99 83C EAA 207 F55 AFF 096 803 CB9 183 355 BC4', '33C FFB 7BE 1A9 793 68E 83C EAA 8ED F55 AFF 0E4 803 CB9 C23 355 BC4 B8B', 'FFB 7BE 0A5 793 68E 241 EAA 8ED 596 AFF 0E4 014 CB9 C23 BC5 BC4 B8B 0DB']
[[['6BC' 'F2B' '773']
  ['597' 'E8E' '206']
  ['2EA' 'BB7' '63E']]

 [['35F' 'B50' '373']
  ['DB1' '22A' '0DA']
  ['87D' 'A61' 'BEE']]

 [['AD1' '9FC' '248']
  ['5D7' '50B' '840']
  ['458' 'F7D' '021']]]


In [16]:
from scripts.nn_layer import conv_layer
inp_image = enc_tensor3(name='conv_in', size=(32, 50, 50))
out_image = enc_tensor3(name='conv_out', size=(1, 41, 41))
weight = (np.random.uniform(size=(32, 32,10,10))> 0.5).astype(int)
conv_lyr = conv_layer(name='conv_nn', inputs=inp_image, outputs=out_image, weight=weight)
nb = reduce(lambda x,y : x*y, inp_image.size)
layer_obj = circuit('linear_layer', conv_lyr, const_inputs=[])
layer_obj.write_file(filename='./test_cnn.sheep')


KeyboardInterrupt: 

In [ ]:
samples = 1
processing_time = np.zeros(samples)
for idx in tqdm(range(0, samples)):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb*1)> 0.5))
    inp_dict = inp_image.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test_cnn.sheep',inputs_file,"bool","TFHE",eval_strategy='parallel')
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    #out_vars = out_image.get_variables()
    #xor_val = (weight == np.asarray([x for x in inp_arr_val]))
    #print(xor_val)
    #out_bit_vec = np.asarray([int(results['Outputs'][var]) for var in out_vars])
    #true_bit_vec = (np.sum(np.asarray(xor_val), axis=1)>nb/2).astype(int)
    #assert np.alltrue(out_bit_vec == true_bit_vec)
print(processing_time.mean())